# Creating New Variables in orders_products_merged

In [7]:
# importing libraries

import pandas as pd
import numpy as np
import os
import janitor

In [8]:
# loading in data

path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/clean_data'
opm = pd.read_pickle(os.path.join(path, 'orders_products_merged.pkl'))
df = opm[:1000000]

In [9]:
#opm = opm.drop(columns='Unnamed: 0')

In [10]:
opm.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0


### creating price range descriptions

In [11]:
def price_label(row):

    if row['prices'] <= 5:
        return 'low-range-product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'mid-range product' 
    elif row['prices'] > 15:
        return 'high range' 
    else: return 'not enough data'

In [12]:
df['price_range'] = df.apply(price_label, axis = 1) # axis 1 = rows

/var/folders/6p/dg2x5f3j5clbf9xgg9xqfxjh0000gn/T/ipykernel_59463/4224014922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis = 1) # axis 1 = rows


In [13]:
df.price_range.value_counts()

mid-range product    756450
low-range-product    243550
Name: price_range, dtype: int64

In [14]:
# doing the same thing but using loc on the full data set
opm.loc[opm['prices'] > 15, 'price_range_loc'] = 'high-range product'

In [15]:
opm.loc[(opm['prices'] <= 15) & (opm['prices'] > 5), 'price_range_loc'] = 'mid-range product'

In [16]:
opm.loc[opm['prices'] <= 5, 'price_range_loc'] = 'low-range product'

### creating for loop to display busiest day

In [17]:
result = []

for value in opm['order_dow']:
    if value == 0:
        result.append('busiest day')
    elif value == 4:
        result.append('least busy day')
    else: 
        result.append('regular busy day')

In [18]:
#opm['busiest_day'] = result
#opm['busiest_day'].value_counts(dropna = False)

## updating busiest day col

In [19]:
opm.order_dow.value_counts()

0    6209632
1    5665830
6    4500246
2    4217766
5    4209449
3    3844096
4    3787193
Name: order_dow, dtype: int64

In [20]:
opm = opm.case_when(
    ((opm.order_dow == 0) | (opm.order_dow == 1)), 'busiest days',
    ((opm.order_dow == 3) | (opm.order_dow == 4)), 'slowest days',
    'regular days',
    column_name = 'busiest_days'
)


In [21]:
# cheacking the values for this new col for accuracy
opm.busiest_days.value_counts()

regular days    12927461
busiest days    11875462
slowest days     7631289
Name: busiest_days, dtype: int64

## creating col to show busiest time periods of the day

In [22]:
list_of_top_hours = opm.order_hour_of_day.value_counts().index.tolist()

In [23]:
opm.loc[opm['order_hour_of_day'].isin(list_of_top_hours[0:8]), 'busiest_period_of_day'] = 'most orders'
opm.loc[opm['order_hour_of_day'].isin(list_of_top_hours[8:16]), 'busiest_period_of_day'] = 'average orders'
opm.loc[opm['order_hour_of_day'].isin(list_of_top_hours[16:24]), 'busiest_period_of_day'] = 'fewest orders'
opm.loc[~opm['order_hour_of_day'].isin(list_of_top_hours), 'busiest_period_of_day'] = 'not enough data'

In [24]:
opm.busiest_period_of_day.value_counts()

most orders       21137262
average orders    10006752
fewest orders      1290198
Name: busiest_period_of_day, dtype: int64

In [25]:
# exporting df

path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/clean_data'
opm.to_pickle(os.path.join(path, 'opm.pkl'))